In [2]:
from fdh_gallica import Periodical, Search, Document
from fdh_gallica.parallel_process import iiif_urls_for_documents
from tqdm.autonotebook import tqdm
import json
import pandas as pd
from pandas.io.json import json_normalize
import itertools
from itertools import chain
from itertools import compress
import re
import wikipedia
from bs4 import BeautifulSoup
import requests
from SPARQLWrapper import SPARQLWrapper, XML

C:\Users\haeeun\Anaconda3\lib\site-packages\tqdm\autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
%matplotlib inline

get all images with relevant query

# Fetch BnF description

In [18]:
def reorder_names(x):
    names_order = {1:[0],2:[1,0], 3:[2,0,1], 4:[3,0,1,2], 5:[4,0,1,2,3]}
    s = x.split(',')
    m = names_order[len(s)]
    return ' '.join([y.strip() for x,y in sorted(zip(m,s))])
    

In [34]:
names_df = pd.read_pickle('data/id_name.pkl')

In [24]:
names_df = names_df.rename(columns={'name':'raw_name'})
names_df['name'] = names_df.raw_name.apply(lambda x: x.split('(')[0].strip())
names_df['ordered_name'] = names_df.name.apply(reorder_names)
names_df['sup'] =  names_df.raw_name.apply(lambda x: str(re.findall('\(\S+\)',x)).strip("['']"))

In [98]:
def sparql_str() :
    pass

In [366]:
def sparql_uri(x, option='name'):
    if option == 'name':
        sparql_str.__doc__ = 'PREFIX foaf: <http://xmlns.com/foaf/0.1/>    SELECT ?pers     WHERE {    ?pers foaf:name "%s".    }    LIMIT 200' % (x)
    elif option == 'familyName':        
        sparql_str.__doc__ = 'PREFIX foaf: <http://xmlns.com/foaf/0.1/>    SELECT ?pers     WHERE {    ?pers foaf:familyName "%s".    }    LIMIT 200' % (x)        
    elif option == 'givenName':
        pass
        
    sparql = SPARQLWrapper("http://data.bnf.fr/sparql")
    sparql.setQuery(sparql_str.__doc__)
    sparql.setReturnFormat(XML)
    results = sparql.query().convert()
    xml_ = results.toxml()
    return list(map(lambda x: x[5:-6], re.findall("<uri>\S*</uri>", xml_)))

In [389]:
def get_bnf_description(links):
    result = {}

    for link in links:
        try:
            page = requests.get(link)
            soup = BeautifulSoup(page.content, 'html.parser')
            try:
                bnf_text = soup.find("div", {"itemprop": "description"}).get_text()
            except:
                bnf_text = ''
            
            bnf_title = soup.find("title").get_text()
            result[bnf_title] = (bnf_text, link)

        except:
            pass
            
        
    return result

In [410]:
bnf_reset = False

if bnf_reset:
    
    bnf_namelist = names_df.ordered_name#[0:60]
    bnf_name_error = []
    bnf_not_found = []
    bnf_link = {}
    
    for i in tqdm(bnf_namelist):
        l = i.split()
        try:
            s = sparql_uri(i)
            
            if s == []:
                #try by familyname
                s = sparql_uri(l[0], option='familyName')
                
                if s == []:
                    s = sparql_uri(l[-1], option='familyName')
                    
                    if s == []:
                        s = sparql_uri(i, option='familyName')
                        
                        if s == []:
                            bnf_not_found3.append(i)
                        else:
                            pass
                    else:
                        pass
                else:
                    pass
            else:
                pass
                      
            bnf_link[i] = [s]
        except:
            bnf_name_error.append(i)
            
    with open('bnf_link.json', 'w') as fp:
        json.dump(bnf_link, fp)
        
    bnf_df_ = pd.DataFrame.from_dict(bnf_link, orient='index', columns=['links'])
    bnf_df_['bnf'] = bnf_df_.links.apply(get_bnf_description)
    bnf_df_ = bnf_df_.reset_index().rename(columns={'index':'ordered_name'})
    bnf_df_.to_pickle('data/bnf_text_raw.pkl')
# takes about 20 minutes

In [31]:
import numpy as np

def handle_year(years):
    try:
        byear = years[0]
        dyear = years[1]

        try:
            byear =  int(byear)
        except:
            byear =  1855

        try:
            dyear =  int(dyear)
        except:
            dyear =  1910
        return  min(1910, dyear) - max(1855, byear)

    except:
        return 0
    
        
        

def bnf_match(keys):
    activity = range(1855, 1910)
    
    
#    years = list(map(lambda x: re.findall('1[.89].+-1[.89].+',x),keys))
    years = list(map(lambda x: str(re.findall('1[89].+-1[89].+',x)).strip("['()']"), keys))

    years_list = list(map(lambda x: handle_year(x.split('-')),years))
    
    
    
#    keys_new = list(compress(keys, map(lambda x: x != '',years)))
    
        
    try :
        return keys[np.argmax(years_list)]
    except:
        return np.nan


In [738]:
with open('data/bnf_name_error.json', 'w') as fp:
    json.dump(bnf_name_error, fp)
with open('data/bnf_not_found.json', 'w') as fp:
    json.dump(bnf_not_found, fp)

In [107]:
with open('bnf_link.json','r') as fp:
    bnf_link =  json.load(fp)

In [3]:
bnf_df_ = pd.read_pickle('data/bnf_text_raw.pkl')

In [26]:
bnf_df_merge = pd.merge(bnf_df_, names_df[['raw_name','ordered_name','sup']], on='ordered_name')

In [28]:
bnf_df_merge['names'] = bnf_df_merge.bnf.apply(lambda x: list(x.keys()))
bnf_df_merge['title'] = bnf_df_merge.apply(lambda x:str(x.ordered_name + ' ' + x.sup).strip(), axis=1)
bnf_df_merge['key_title'] = bnf_df_merge.apply(lambda x: list(compress(x.names, list(map(lambda y: x.title in y, x.names)))), axis=1)
bnf_df_merge['key_name'] = bnf_df_merge.apply(lambda x: list(compress(x.names, list(map(lambda y: x.ordered_name in y, x.names)))), axis=1)
bnf_df_merge['key_sup'] = bnf_df_merge.apply(lambda x: list(compress(x.names, list(map(lambda y: x.sup in y, x.names)))), axis=1)
bnf_df_merge['key'] = bnf_df_merge.apply(lambda x: list(set.intersection(set(x.key_title), set(x.key_name), set(x.key_sup))), axis=1)

In [32]:
bnf_df_merge['key_match'] = bnf_df_merge.key.apply(bnf_match)#.apply(lambda x: len(x)).value_counts(ascending=False)
bnf_df_merge['description'] = bnf_df_merge.dropna(axis=0).apply(lambda x: x.bnf[x.key_match],axis=1)
#bnf_df_merge[['ordered_name','description']].to_pickle('data/bnf_description.pkl')#.description.dropna(axis=0)

In [ ]:
bnf_summary = bnf_df_merge[['raw_name','description']]
bnf_summary['bnf_note'] = bnf_df_merge.description.apply(lambda x: x[0])
bnf_summary['bnf_link'] = bnf_df_merge.description.apply(lambda x: x[1])
#bnf_summary.to_pickle('data/bnf_summaries')